In [10]:
from pandas import DataFrame, Series
import pandas as pd
import requests

In [1]:
def fetch(url:str, **kwargs):
    resp = requests.get(f"https://api.bithumb.com/v1{url}", headers={"accept": "application/json"})
    json = resp.json()
    if len(json) == 1:
        return Series(json[0], **kwargs)
    else:
        return DataFrame(json, **kwargs)

In [25]:
class Coin:

    rename = {
        "candle_date_time_kst": "datetime",
        "opening_price": "open",
        "high_price": "high",
        "low_price": "low",
        "trade_price": "close",
        "candle_acc_trade_price": "amount",
        "candle_acc_trade_volume": "volume",
        "trade_volume": 'volume',
        "ask_bid": "quote",
        "datetime": "datetime"
    }

    def __init__(self, ticker:str):
        self.ticker = ticker
        return

    def __repr__(self):
        return repr(self.snapShot)
    
    def snapShot(self) -> Series:
        return query(f"/ticker?markets={self.ticker}", name=self.ticker)

    def ohlcv(self, period:str='d', *args, **kwargs) -> DataFrame:
        period = {'d':'days', 'min':'minutes', 'w':'weeks', 'm':'months'}[period.lower()]
        if period == 'minutes':
            unit = args[0] if args else kwargs.get("unit", 60)
            query = f'/candles/{period}/{unit}?market={self.ticker}&count=200'
        else:
            query = f'/candles/{period}?market={self.ticker}&count=200'
        
        data = fetch(query)
        cols = {k:v for k, v in self.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        data['volume'] = data['volume'] * 1e+6
        return data

    def execution(self, count:int=100):
        data = fetch(f'/trades/ticks?market={self.ticker}&count={min(count, 500)}')
        data['datetime'] = pd.to_datetime(data['trade_date_utc'].astype(str) + ' ' + data['trade_time_utc'].astype(str))
        data['datetime'] = data['datetime'] + pd.Timedelta(hours=9)
        cols = {k:v for k, v in self.rename.items() if k in data.columns}
        
        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        return data


coin = Coin('KRW-BTC')
# coin.snapShot()
# coin.ohlcv('min', 30)
coin.execution()

,close,volume,quote
datetime,,,
2025-11-27 18:01:03,136730000,0.007300,ASK
2025-11-27 18:01:03,136730000,0.002077,ASK
2025-11-27 18:01:02,136736000,0.003000,BID
2025-11-27 18:01:01,136725000,0.002167,ASK
2025-11-27 18:01:01,136736000,0.001889,BID
...,...,...,...
2025-11-27 17:59:21,136721000,0.000953,BID
2025-11-27 17:59:21,136706000,0.000510,BID
2025-11-27 17:59:18,136695000,0.006256,ASK


In [ ]:
class Coins:

    url = "https://api.bithumb.com/v1/market/all?isDetails=true"
    headers = {"accept": "application/json"}
    
    def __init__(self, currency:str=''):
        get = DataFrame(requests.get(url=self.url, headers=self.headers).json())
        get = get.rename(columns={'market':'ticker', 'korean_name':'kor', 'english_name':'eng', 'market_warning':'warning'})
        get = get[["ticker", "eng", "warning", "kor"]]
        get["warning"] = get["warning"].replace("NONE", None)
        if currency:
            get = get[get['ticker'].str.startswith(currency)]
        # super().__init__(get)
        return

    def warnings(self):
        q = query(url="https://api.bithumb.com/v1/market/virtual_asset_warning")
        return q

coins = Coins(currency='KRW')
# coins = coins[coins.warning.isna()]
# coins
coins.warnings()
